In [40]:
import pandas, copy, json, bz2

from tqdm import tqdm

pandas.options.display.max_columns=999

# Creating the COGUK dataset

First, let's load in the COGUK metadata file that is publicly available via

`wget https://cog-uk.s3.climb.ac.uk/phylogenetics/latest/cog_metadata.csv
`

Note that we can infer `cogid` from the `sequence_name` and it is not guaranteed unique, so we shall keep the last.

In [2]:
cog_metadata=pandas.read_csv('cog_metadata-20210624.csv.gz')

def split_sequence_name(row):
    cols=row['sequence_name'].split('/')
    if len(cols)==3:
        return(cols[1])
    else:
        return(None)

# rename the country to avoid a conflict with the ENA dataset later
cog_metadata.rename(columns={'country':'cog_country'},inplace=True)

# infer the cogid
cog_metadata['cogid']=cog_metadata.apply(split_sequence_name,axis=1)        

# set as the index and de-duplicate
cog_metadata.set_index('cogid',inplace=True,verify_integrity=False)
cog_metadata=cog_metadata.loc[~cog_metadata.index.duplicated(keep='first')]
cog_metadata.reset_index(inplace=True)
cog_metadata.set_index('cogid',inplace=True,verify_integrity=True)

print("There are "+str(len(cog_metadata))+" rows in the dataset")

cog_metadata[:3]

/opt/local/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


There are 506543 rows in the dataset


,sequence_name,cog_country,adm1,is_pillar_2,sample_date,epi_week,lineage,lineages_version,lineage_conflict,lineage_ambiguity_score,scorpio_call,scorpio_support,scorpio_conflict,t1001i,n501y,d614g,del_21765_6,y453f,del_1605_3,a222v,n439k,mutations,p681h,q27stop,e484k,p323l
cogid,,,,,,,,,,,,,,,,,,,,,,,,,,
PHEC-L307LF63,Northern_Ireland/PHEC-L307LF63/2021,UK,UK-NIR,N,2021-06-15,77,B.1.617.2,PLEARN-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0,0.0,T,N,G,ref,Y,ref,A,N,synSNP:G1186C|synSNP:C3037T|orf1ab:A1306S|orf1...,R,Q,E,L
PHEC-L307LF81,Northern_Ireland/PHEC-L307LF81/2021,UK,UK-NIR,N,2021-06-15,77,B.1.617.2,PLEARN-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0,0.0,T,N,G,ref,Y,ref,A,N,synSNP:C3037T|orf1ab:A1306S|orf1ab:P2046L|orf1...,R,Q,E,L
PHEC-L307LF45,Northern_Ireland/PHEC-L307LF45/2021,UK,UK-NIR,N,2021-06-15,77,B.1.617.2,PLEARN-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0,0.0,T,N,G,ref,Y,ref,A,N,orf1ab:P309L|synSNP:C1267T|synSNP:C3037T|orf1a...,R,Q,E,L


But we don't have the ENA `run_accession` which we need to link to the FASTQ files in object store, nor do we know what the sequencing platform was, which we need to choose the right flow in SP3, so let's load in a list from the ENA and again we can infer `cogid` from the `submitted_ftp` file.

To get the latest TSV issue the below in a terminal; be warned that unless you subset the fields it will be >2GB; either way best to compress.

`curl "https://www.ebi.ac.uk/ena/portal/api/filereport?accession=PRJEB37886&result=read_run&fields=study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,instrument_platform,country,submitted_ftp,read_count,base_count&limit=1000000&format=tsv&download=true" > cog-ena-20210624.tsv`

In [3]:
ena_sample=pandas.read_csv('ena_samples_all_fields.tsv',sep='\t')
ena_sample[:2]

,study_accession,secondary_study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,submission_accession,tax_id,scientific_name,instrument_platform,instrument_model,library_name,library_layout,nominal_length,library_strategy,library_source,library_selection,read_count,base_count,center_name,first_public,last_updated,experiment_title,study_title,study_alias,experiment_alias,run_alias,fastq_bytes,fastq_md5,fastq_ftp,fastq_aspera,fastq_galaxy,submitted_bytes,submitted_md5,submitted_ftp,submitted_aspera,submitted_galaxy,submitted_format,sra_bytes,sra_md5,sra_ftp,sra_aspera,sra_galaxy,cram_index_ftp,cram_index_aspera,cram_index_galaxy,sample_alias,broker_name,nominal_sdev,first_created,sample_description,parent_study,sample_accession.1,accession,secondary_sample_accession.1,bio_material,cell_line,cell_type,collected_by,collection_date,country,cultivar,culture_collection,description,dev_stage,ecotype,environmental_sample,first_public.1,germline,identified_by,isolate,isolation_source,location,mating_type,serotype,serovar,sex,submitted_sex,specimen_voucher,strain,sub_species,sub_strain,tissue_lib,tissue_type,variety,tax_id.1,scientific_name.1,sample_alias.1,checklist,center_name.1,depth,elevation,altitude,environment_biome,environment_feature,environment_material,temperature,salinity,sampling_campaign,sampling_site,sampling_platform,protocol_label,project_name,host,host_tax_id,host_status,host_sex,submitted_host_sex,host_body_site,host_gravidity,host_phenotype,host_genotype,host_growth_conditions,environmental_package,investigation_type,experimental_factor,sample_collection,sequencing_method,target_gene,ph,broker_name.1,sample_title,sample_material,taxonomic_identity_marker,assembly_quality,assembly_software,taxonomic_classification,completeness_score,contamination_score,binning_software,lat,lon,sample_capture_status,collection_date_submitted,submission_tool
0,PRJEB37886,ERP121228,SAMEA6791711,ERS4519383,ERX4078237,ERR4082025,ERA2524107,2697049,Severe acute respiratory syndrome coronavirus 2,OXFORD_NANOPORE,MinION,NaN,SINGLE,NaN,AMPLICON,VIRAL RNA,PCR,211388,103672824,University of Cambridge,2020-04-29,2020-04-29,MinION sequencing; COG-UK/CAMB-7374F/CAMB:2020...,COVID-19 Genomics UK (COG-UK) consortium,ena-STUDY-COVID-19 Genomics UK Consortium-18-0...,COG-UK/CAMB-7374F/CAMB:20200404_1455_MN30591_F...,COG-UK/CAMB-7374F/CAMB:20200404_1455_MN30591_F...,91988098,28d6f7213833a6e5025fb4d42767e011,ftp.sra.ebi.ac.uk/vol1/fastq/ERR408/005/ERR408...,fasp.sra.ebi.ac.uk:/vol1/fastq/ERR408/005/ERR4...,ftp.sra.ebi.ac.uk/vol1/fastq/ERR408/005/ERR408...,108722728,18f53e799c25060ad74ae240c253f3a8,ftp.sra.ebi.ac.uk/vol1/run/ERR408/ERR4082025/C...,fasp.sra.ebi.ac.uk:/vol1/run/ERR408/ERR4082025...,ftp.sra.ebi.ac.uk/vol1/run/ERR408/ERR4082025/C...,BAM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COG-UK/CAMB-7374F,COVID-19 Genomics UK Consortium,NaN,2020-04-29,COG-UK/CAMB-7374F,PRJEB39908,SAMEA6791711,SAMEA6791711,ERS4519383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MinION sequencing; COG-UK/CAMB-7374F/CAMB:2020...,NaN,NaN,False,2020-04-29,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2697049,Severe acute respiratory syndrome coronavirus 2,COG-UK/CAMB-7374F,ERC000011,University of Cambridge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COVID-19 Genomics UK Consortium,COG-UK/CAMB-7374F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PRJEB37886,ERP121228,SAMEA6791712,ERS4519384,ERX4078238,ERR4082026,ERA2524109,2697049,Severe acute respiratory syndrome coronavirus 2,OXFORD_NANOPORE,MinION,NaN,SINGLE,NaN,AMPLICON,VIRAL RNA,PCR,427553,210103657,University of Cambridge,2020-04-29,2020-04-29,MinION sequencing; COG-UK/CAMB-7375E/CAMB:2020...,COVID-19 Genomics UK (COG-UK) consortium,ena-STUDY-COVID-19 Genomics UK Consortium-18-0...,COG-UK/CAMB-7375E/CAMB:20200404_1455_MN30591_F...,COG-UK/CAMB-7375E/CAMB:20200404_1455_MN30591_F...,186032909,811cf092

In [4]:
cog_ena=pandas.read_csv('cog_ena-20210624.tsv.gz',sep='\t')

def create_cogid(row):
    if isinstance(row['submitted_ftp'],str):
        return(row['submitted_ftp'].split('/')[-1].split('.')[0])
    else:
        return(None)

# rename the country to avoid a conflict with the metadata dataset later
cog_ena.rename(columns={'country':'ena_country'},inplace=True)

# filter down to Illumina and ONT only (excludes Ion Torrent)
cog_ena=cog_ena.loc[cog_ena.instrument_platform.isin(['ILLUMINA','OXFORD_NANOPORE'])]

# infer the cogid
cog_ena['cogid']=cog_ena.apply(create_cogid,axis=1)

# remove any samples with no cogid
cog_ena=cog_ena.loc[cog_ena.cogid.notna()]

# set as the index and de-duplicate
cog_ena.set_index('cogid',inplace=True,verify_integrity=False)
cog_ena=cog_ena.loc[~cog_ena.index.duplicated(keep='first')]
cog_ena.reset_index(inplace=True)
cog_ena.set_index('cogid',inplace=True,verify_integrity=True)

print("There are "+str(len(cog_ena))+" rows in the dataset")

cog_ena[:3]

There are 496763 rows in the dataset


,study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,instrument_platform,ena_country,submitted_ftp,read_count,base_count
cogid,,,,,,,,,,
CAMB-7374F,PRJEB37886,SAMEA6791711,ERS4519383,ERX4078237,ERR4082025,OXFORD_NANOPORE,NaN,ftp.sra.ebi.ac.uk/vol1/run/ERR408/ERR4082025/C...,211388.0,103672824.0
CAMB-7375E,PRJEB37886,SAMEA6791712,ERS4519384,ERX4078238,ERR4082026,OXFORD_NANOPORE,NaN,ftp.sra.ebi.ac.uk/vol1/run/ERR408/ERR4082026/C...,427553.0,210103657.0
CAMB-7376D,PRJEB37886,SAMEA6791713,ERS4519385,ERX4078239,ERR4082027,OXFORD_NANOPORE,NaN,ftp.sra.ebi.ac.uk/vol1/run/ERR408/ERR4082027/C...,374754.0,183823695.0


Now let's do an inner join so every row has a unique `cogid` and an ENA `run_accession`

In [51]:
cog=cog_metadata.join(cog_ena,how='inner')

In [52]:
print("This gives us "+str(len(cog))+" COGUK samples with an ENA accession number")

print("There are " +str(len(cog.loc[cog.lineage.isna()]))+" samples with no PANGO lineage and " +str(len(cog.loc[cog.scorpio_call.isna()]))+ " samples with no SCORPIO lineage")

This gives us 455352 COGUK samples with an ENA accession number
There are 0 samples with no PANGO lineage and 169437 samples with no SCORPIO lineage


Now let's match to what is in the `ena` bucket on object store so we can make sure we are only asking for run accessions that have fastq files in the bucket.

In [53]:
with bz2.open('oci-ena-bucket-20210625.json.bz2') as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()
    
jsonPayload=jsonObject['data']    

In [54]:
ena_accessions=set()

for i in tqdm(jsonPayload):
    
    fastaFile=i['name']
    
    if "_" in fastaFile:
        era=fastaFile.split('_')[0]
    else:
        era=fastaFile.split('.')[0]
    if era not in ena_accessions:
        ena_accessions.add(era)

print("There are "+str(len(ena_accessions))+" unique ENA run accessions stored in the ena bucket in object store")

100%|██████████| 764990/764990 [00:00<00:00, 812165.37it/s]

There are 308136 unique ENA run accessions stored in the ena bucket in object store


In [55]:
cog['in_ena_bucket']=cog.run_accession.isin(ena_accessions)
cog.in_ena_bucket.value_counts()

True     228773
False    226579
Name: in_ena_bucket, dtype: int64

In [56]:
cog.to_csv('cog_dataset-20210624.csv.gz')